In [ ]:
# only z is quantized
# for f0 and ld different heads in the rnn is used.

In [ ]:
tfkl = tf.keras.layers

seqlen = 1000
dim_code = 18

# RNN model with 3 heads
inputs = tfkl.Input(batch_shape=(batch_size, None, dim_code))
x = tfkl.GRU(512, return_sequences=True, stateful=True)(inputs)
# x = tfkl.GRU(512, return_sequences=True, stateful=True)(x)
x = tfkl.Dense(256, tf.nn.relu)(x) 
# 3 seperate output layers
f0_output = tfkl.Dense(128)(x)
ld_output = tfkl.Dense(121)(x)
z_output = tfkl.Dense(64)(x)
    
model_rnn = tf.keras.Model(inputs=inputs, outputs=[f0_output, ld_output, z_output], name='Functional-api-RNN')

train_steps = 200000
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)
lr = tf.optimizers.schedules.PolynomialDecay(0.0003, train_steps, 0.00001)
opt = tf.optimizers.Adam(lr)

In [ ]:
@tf.function
def train(ind_batch, code_batch):
    targets_f0 = tf.cast(code_batch[:, 1:, 0:1] * 127, tf.dtypes.int64) # 128 MIDI pitches for f0_scaled targets
    targets_f0 = tf.reshape(targets_f0, [-1,seqlen-1])
    targets_ld = ind_batch[:, 1:, 0:1] # this is loudness_db used as labels for ld_scaled
    targets_ld = tf.reshape(targets_ld, [-1,seqlen-1]) * (-1) # sign of the label altered from '-' to '+'
    targets_z = ind_batch[:, 1:, 1:]
    targets_z = tf.reshape(targets_z, [-1,seqlen-1])
    inp = code_batch[:, :-1]
    with tf.GradientTape() as tape:
        out = model_rnn(inp)
        xent_f0 = loss(targets_f0, out[0])
        xent_ld = loss(targets_ld, out[1])
        xent_z = loss(targets_z, out[2])
        xent = (xent_f0 + xent_ld + xent_z) / 3.
    
    grads = tape.gradient(xent, model_rnn.trainable_variables)
    opt.apply_gradients(zip(grads, model_rnn.trainable_variables))
    return xent

losses = []
for st, (inds, codes) in enumerate(code_data_ready):
    if st > train_steps:
        break
        
    model_rnn.reset_states()
    xent = train(inds, codes)
    losses.append(xent)
    if not st % 1000:
        print(st, xent)
losses = tf.concat(losses, axis=0)

In [ ]:
def generate_rnn():
    model_rnn.reset_states()
    gen_f0 = tf.convert_to_tensor(np.random.randint(128, size=(batch_size, 1)))
    gen_f0 = tf.cast(tf.reshape(gen_f0/127, (batch_size, 1, 1)), tf.dtypes.float32)
    gen_ld = tf.convert_to_tensor(np.random.randint(121, size=(batch_size, 1)))
    gen_ld = tf.cast(tf.reshape((-gen_ld/120)+1, (batch_size, 1, 1)), tf.dtypes.float32) # (features['loudness_db'] / LD_RANGE) + 1.0
    gen_z = tf.convert_to_tensor(np.random.randint(64, size=(batch_size, 1)))
    gen_z = tf.gather(codebook, gen_z)
    out = tf.concat((gen_f0, gen_ld, gen_z), axis=-1)
    collected = [out] 
    for t in range(seqlen-1):
        out = model_rnn(out)
        gen_f0 = tfd.Categorical(logits=out[0]).sample()
        gen_f0 = tf.cast(tf.reshape(gen_f0/127, (batch_size, 1, 1)), tf.dtypes.float32)
        gen_ld = tfd.Categorical(logits=out[1]).sample()
        gen_ld = tf.cast(tf.reshape((-gen_ld/120)+1, (batch_size, 1, 1)), tf.dtypes.float32)
        gen_z = tfd.Categorical(logits=out[2]).sample()
        gen_z = tf.gather(codebook, gen_z)
        out = tf.concat((gen_f0, gen_ld, gen_z), axis=-1)
        collected.append(out)
    collected = tf.concat(collected, axis=1)
    gen_f0, gen_ld, gen_z = tf.split(collected, [1,1,16], axis=-1)
    return gen_f0, gen_ld, gen_z

In [ ]:
# Generate Audio

fs, ld, z = generate_rnn()
fh = fs*127
fh = 2**((fh - 69) / 12) * 440

back_to_decoder = {}
back_to_decoder['f0_hz'] = fh
back_to_decoder['f0_scaled'] = fs
back_to_decoder['ld_scaled'] = ld
back_to_decoder['z'] = z

audio_gen = model_vq.decode(back_to_decoder)
# some audio_gen outputs are nan